# Evaluación Reconocimiento de Patrones
Ariana Espinoza

<style>
    .imagen-ajustada {
        width: 900px;
        height: 280px;
        border-radius: 20px;
    }
</style>

<img src="Arboles.jpg" alt="Una imagen" class="imagen-ajustada">

Para el ejercicio del examen se pidió utilizar el conjunto de datos el cual tiene 54 características, las cuales se obtuvieron de observaciones de terrenos en el bosque. Esto se realizó con el fin de predecir el tipo de cubierta del terreno. Los datos están en forma cruda (no escalados) y contienen columnas binarias (0 o 1) de datos para variables cualitativas independientes (áreas silvestres y tipos de suelo). Estas áreas representan bosques con mínimas perturbaciones causadas por el hombre, de modo que los tipos de cubierta forestal existentes son más el resultado de procesos ecológicos que de prácticas de manejo forestal.

Las áreas están designadas de la siguiente manera: Neota (área 2) probablemente tiene el valor de elevación promedio más alto de las 4 áreas silvestres. Rawah (área 1) y Comanche Peak (área 3) tendrían un valor de elevación medio más bajo, mientras que Cache la Poudre (área 4) tendría el valor de elevación medio más bajo. En cuanto a las principales especies de árboles en estas áreas, Neota tendría spruce/fir (tipo 1), mientras que Rawah y Comanche Peak probablemente tendrían lodgepole pine (tipo 2) como su especie principal, seguido por spruce/fir y aspen (tipo 5). Cache la Poudre tendería a tener Ponderosa pine (tipo 3), Douglas-fir (tipo 6) y cottonwood/willow (tipo 4).   

Para el ejercicio del examen se pidió hacer un estudio evaluando el rendimiento de SVM, K-vecinos, árboles de decisión, Random Forest, Redes neuronales y gradient boosting classifier. Reportar evaluaciones de los clasificadores individuales y de un ensamble usando sklearn.metrics.classification_report y sklearn.ensemble.VotingClassifier, comentar sobre los resultados.

## Importando Librerías Necesarias

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Cargando los datos

Para acceder a los datos decidí traerlos por medio de un URL para evitar complicaciones al momento de acceder a ellos, sin embargo me di cuenta que tardaba un poco al momento de extraerlos, por lo que decidí guardarlo en un archivo .CSV para que fuera más fácil llamarlos. Además se le pone nombre a cada columna para que no aparezca por números.

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"
column_names = ["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
                "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
                "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
                "Horizontal_Distance_To_Fire_Points"] + [f"Wilderness_Area_{i}" for i in range(4)] + \
                [f"Soil_Type_{i}" for i in range(40)] + ["Cover_Type"]
datos = pd.read_csv(url, header=None, names=column_names)

In [3]:
# Con base a los datos que se trajeron por medio de la URL, se crea el archivo .CSV
datos.to_csv("covertype_datos.csv", index=False)

In [4]:
# Se lee el archivo .CSV para utilizarlo con el entrenamiento de datos.
datosArchivo = pd.read_csv("covertype_datos.csv")

In [5]:
# Para ver el tipo de datos que arroja el conjunto de datos, se utiliza el método head(), el cual trae los primeros cinco resultados, pudiendo ver que ningún dato esta faltante; 
# también se puede apreciar la clsificación de las clases.
datosArchivo.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


## Separación de Datos

Para pasar al entrenamiento de datos se requiere previamente haber entrenado los datos. Como primer paso los datos se dividen en X y Y, basandosé en si los datos son dependientes o indepedientes, posterior a eso se dvididen los datos en 70% entrenamiento y 30% de prueba, finalmente pasan al escalado.

In [6]:
# Se separan las características y las etiquetas
X = datosArchivo.drop("Cover_Type", axis=1)
y = datosArchivo["Cover_Type"]

In [7]:
# Se dividen los datos en conjuntos de 70% de entrenamiento y 30% de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
# Se escalan los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Entrenando los modelos

Para evaluar los resultados de forma individual, decidí poner cada uno de forma individual en lugar de hacer un ciclo que requiriera ponerlos todos. Esto se debe a que tardaba demasiado y pensé que se verían más limpios los resultados de esta manera, algo que hay que mencionar es que al final de cada modelo se utiliza el método de classification_report, el cual imprime un informe de clasificación que compara las etiquetas reales del conjunto de prueba y_test con las etiquetas predichas y_pred_svm. Este informe incluye métricas como precisión, recall, F1-score y soporte.

En el caso del resultado obtenido por medio de SVM se puede decir que no le fue tan bien al obtener 79% de accuracy, además de que en otros no le fue mejor.

In [9]:
# Entrena y evalúa el modelo de Máquinas de Vectores de Soporte (SVM)
svm_model = SVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
print("Classification Report for SVM:")
print(classification_report(y_test, y_pred_svm))
print("\n" + "-"*60 + "\n")

Classification Report for SVM:
              precision    recall  f1-score   support

           1       0.80      0.75      0.78     63556
           2       0.79      0.86      0.83     85078
           3       0.74      0.87      0.80     10638
           4       0.74      0.43      0.55       795
           5       0.80      0.12      0.20      2941
           6       0.67      0.40      0.50      5227
           7       0.85      0.74      0.79      6069

    accuracy                           0.79    174304
   macro avg       0.77      0.60      0.63    174304
weighted avg       0.79      0.79      0.78    174304


------------------------------------------------------------



El resultado obtenido por medio de K-Neighbors fue mejor de lo esperado, ya que obtuvo un resultado de 92% de accuracy, además de que en lo demás saco un puntaje un poco más alto.

In [24]:
# Entrena y evalúa el modelo de K-Neighbors
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
print("Classification Report for K-Neighbors:")
print(classification_report(y_test, y_pred_knn))
print("\n" + "-"*60 + "\n")


Classification Report for K-Neighbors:
              precision    recall  f1-score   support

           1       0.93      0.92      0.93     63556
           2       0.93      0.94      0.94     85078
           3       0.90      0.91      0.90     10638
           4       0.85      0.73      0.78       795
           5       0.84      0.75      0.79      2941
           6       0.82      0.80      0.81      5227
           7       0.94      0.94      0.94      6069

    accuracy                           0.92    174304
   macro avg       0.89      0.86      0.87    174304
weighted avg       0.92      0.92      0.92    174304


------------------------------------------------------------



El resultado obtenido por medio de Decision Tree fue muy bueno, ya que obtuvo un resultado de 93% de accuracy, pero en este caso la métrica permaneció igual en los otros métodos.

In [22]:
# Entrena y evalúa el modelo de Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)
print("Classification Report for Decision Tree:")
print(classification_report(y_test, y_pred_dt))
print("\n" + "-"*60 + "\n")


Classification Report for Decision Tree:
              precision    recall  f1-score   support

           1       0.93      0.93      0.93     63556
           2       0.94      0.94      0.94     85078
           3       0.92      0.93      0.93     10638
           4       0.84      0.84      0.84       795
           5       0.81      0.81      0.81      2941
           6       0.87      0.86      0.87      5227
           7       0.94      0.95      0.94      6069

    accuracy                           0.93    174304
   macro avg       0.90      0.89      0.90    174304
weighted avg       0.93      0.93      0.93    174304


------------------------------------------------------------



El resultado obtenido por medio de Random Forest fue el mejor de todos los clasificadores, ya que obtuvo un resultado de 95% de accuracy, pero algo curioso es que en en la precisión sube a 96 y en la parte de recall baja a 94. Dejando esto del lado, fue el mejor de todos los clasificadores utilizados en la práctica.

In [23]:
# Entrena y evalúa el modelo de Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("Classification Report for Random Forest:")
print(classification_report(y_test, y_pred_rf))
print("\n" + "-"*60 + "\n")


Classification Report for Random Forest:
              precision    recall  f1-score   support

           1       0.96      0.94      0.95     63556
           2       0.95      0.97      0.96     85078
           3       0.94      0.96      0.95     10638
           4       0.92      0.86      0.89       795
           5       0.94      0.75      0.84      2941
           6       0.93      0.89      0.91      5227
           7       0.97      0.95      0.96      6069

    accuracy                           0.95    174304
   macro avg       0.94      0.90      0.92    174304
weighted avg       0.95      0.95      0.95    174304


------------------------------------------------------------



El resultado obtenido por medio de Gradient Boosting no fue bueno, ya que obtuvo un resultado de 77% de accuracy, siendo el resultado más bajo obtenido por medio de los clasificadores .

In [25]:
# Entrena y evalúa el modelo de Gradient Boosting
gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
print("Classification Report for Gradient Boosting:")
print(classification_report(y_test, y_pred_gb))
print("\n" + "-"*60 + "\n")


Classification Report for Gradient Boosting:
              precision    recall  f1-score   support

           1       0.76      0.75      0.75     63556
           2       0.78      0.82      0.80     85078
           3       0.77      0.83      0.80     10638
           4       0.79      0.74      0.77       795
           5       0.75      0.23      0.36      2941
           6       0.66      0.48      0.55      5227
           7       0.87      0.70      0.77      6069

    accuracy                           0.77    174304
   macro avg       0.77      0.65      0.69    174304
weighted avg       0.77      0.77      0.77    174304


------------------------------------------------------------



El resultado obtenido por medio de una red neuronal fue bueno, obteniendo un resultado de 87% de accuracy, algo interesante que se ve es que consiguió una precisión de 90%, 81% de recall y 86% de f1 score, todo muy variado.

In [27]:
# Entrena y evalúa el modelo de Neural Network
nn_model = MLPClassifier(max_iter=300)
nn_model.fit(X_train, y_train)
y_pred_nn = nn_model.predict(X_test)
print("Classification Report for Neural Network:")
print(classification_report(y_test, y_pred_nn))
print("\n" + "-"*60 + "\n")


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Classification Report for Neural Network:
              precision    recall  f1-score   support

           1       0.90      0.81      0.86     63556
           2       0.86      0.92      0.89     85078
           3       0.86      0.87      0.87     10638
           4       0.85      0.75      0.80       795
           5       0.75      0.54      0.63      2941
           6       0.77      0.74      0.75      5227
           7       0.85      0.92      0.89      6069

    accuracy                           0.87    174304
   macro avg       0.83      0.79      0.81    174304
weighted avg       0.87      0.87      0.87    174304


------------------------------------------------------------



Como último paso para la comparativa entre modelos de clasificación, se requiere utilizar un ensamble que abarque todo lo anteriormente utilizado pero al mismo tiempo combinarlos para ver si con esto se obtiene un mejor resultado. El ensamblaje VotingClassifier logro una alta precisión y exactitud global del 92%. Sin embargo, hay algunas clases específicas donde el recall es más bajo, lo que indica que hay margen de mejora en la identificación correcta de todos los casos de esas clases. En general, el uso de múltiples clasificadores ha permitido obtener un desempeño robusto y equilibrado.

In [10]:
# Se crea  el ensamble de VotingClassifier
ensemble = VotingClassifier(estimators=[
    ('svc', SVC()),
    ('knn', KNeighborsClassifier()),
    ('dt', DecisionTreeClassifier()),
    ('rf', RandomForestClassifier()),
    ('gb', GradientBoostingClassifier()),
    ('nn', MLPClassifier(max_iter=300))
], voting='hard')

# Entrena y evalua el ensamble
ensemble.fit(X_train, y_train)
y_pred_ensemble = ensemble.predict(X_test)
print("Classification Report for Voting Ensemble:")
print(classification_report(y_test, y_pred_ensemble))

Classification Report for Voting Ensemble:
              precision    recall  f1-score   support

           1       0.91      0.93      0.92     63556
           2       0.93      0.94      0.93     85078
           3       0.88      0.93      0.90     10638
           4       0.92      0.77      0.84       795
           5       0.95      0.55      0.70      2941
           6       0.92      0.72      0.81      5227
           7       0.97      0.89      0.93      6069

    accuracy                           0.92    174304
   macro avg       0.93      0.82      0.86    174304
weighted avg       0.92      0.92      0.92    174304



El uso de un ensamble de modelos (VotingClassifier) proporcionó la mejor precisión, alcanzando aproximadamente un 92%. Esto demuestra la ventaja de combinar varios modelos para mejorar la precisión y robustez del sistema de clasificación, sin embargo se puede decir que le fue bien pero no tan bien como al clasificador Random Forest. De igual forma se puede decir que cada modelo individual ofreció contribuciones valiosas y mostró puntos fuertes en diferentes aspectos, pero el enfoque de ensamble fue uno de los más efectivo para este conjunto de datos, junto con Random Forest, Decision Tree y K-Neighbors.

En conclusión, el proyecto nos muestra la importancia de evaluar múltiples modelos y enfoques en problemas de clasificación, destacando cómo las técnicas de ensamble pueden mejorar el rendimiento predictivo. La variabilidad en las precisiones refleja la naturaleza diversa y compleja de los datos de cubiertas de terreno, subrayando la necesidad de enfoques robustos y bien calibrados para tareas de clasificación en el ámbito de la ciencia de datos.

Un comentario muy personal es que no pensé que los clasificadores tomarán tanto tiempo en ejecutarse, creo que se debió más que nada a mi computadora, ya que tardo seis horas con unos mninutos en poder ejecutar el ensamble. Además, me parece genial lo del ensamble, jamás lo había utilizado y pienso que dio buen resultado, pero de igual forma siento que otros clasificadores lo hicieron casi tan bueno o mejor que el ensamble, así que sirvió este ejercicio para aprender más de como tratar con clasificadores, compararlos y ver a cual de ellos le fue mejor.